# Goal

There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.
https://gym.openai.com/envs/Taxi-v3/

In [18]:
import numpy as np
import gym
import random
import time
from tqdm import tqdm
from IPython.display import clear_output

In [19]:
env = gym.make("Taxi-v3")

## Q-table

The Q-table is an array with each line corresponding to a specific state, and each column corresponding to a specific action.
We can then refer to that Q-table to select a specific state-action pair.

In [5]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [14]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.98035354, -0.73010001, -3.16883054, -4.11774313,  9.6220697 ,
        -7.45269128],
       [ 2.80500793, -1.96587917, -2.03462204, -0.48524037, 14.11880599,
        -5.39463893],
       ...,
       [-1.06604517,  0.2179915 , -0.99551198, -0.99337936, -3.51140357,
        -4.1971362 ],
       [-2.55285152, -2.57588094, -2.53587877, -2.6410169 , -2.84132105,
        -6.23360338],
       [-0.1       , -0.1       , -0.1       ,  1.96784961, -1.        ,
         0.        ]])

## Learning rate and discount rate

The learning rate decides how fast we should replace the previous q value from the q-table that we are updating. We don't want the q-value to be completely replaced with the new q-value.

The discount rate $ \gamma $ determines the present value of future rewards:

$$
\begin{eqnarray*} 
G_{t} &=&R_{t+1}+\gamma R_{t+2}+\gamma ^{2}R_{t+3}+\cdots \\ &=&\sum_{k=0}^{\infty }\gamma^{k}R_{t+k+1}\text{.} 
\end{eqnarray*}
$$

With a smaller discount rate, the agent will care more about the immediate reward.

In [7]:
num_episodes = 10000
max_steps_per_episode = 4000

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

In [13]:
rewards_all_episodes = []

## Exploration vs Exploitation

To avoid exploiting actions that might not be as rewarding as others actions in the long run, we create an exploration rate threshold variable and an exploration rate variable. When the former one is smaller than the last one, we pick an action from the action space that might not be the one with the highest q value for now. 

The Q-value for the state-action pair in the Q-table is updated using the following formula:

$$
\begin{equation*}
q^{new}\left( s,a\right) =\left( 1-\alpha \right) ~\underset{\text{old value} }{\underbrace{q\left( s,a\right)}\rule[-0.05in]{0in}{0.2in} \rule[-0.05in]{0in}{0.2in}\rule[-0.1in]{0in}{0.3in}}+\alpha \overset{\text{ learned value}}{\overbrace{\left(R_{t+1}+\gamma \max_{a^{^{\prime }}}q\left( s^{\prime },a^{\prime }\right) \right) }} 
\end{equation*}
$$

In [20]:
for episode in tqdm(range(num_episodes)):
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        q_table[state, action] = (1 - learning_rate) * q_table[state, action] + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
    
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    rewards_all_episodes.append(rewards_current_episode)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4410.13it/s]


In [21]:
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  -15.946999999999774
2000 :  7.445999999999958
3000 :  7.34299999999996
4000 :  7.441999999999964
5000 :  7.485999999999968
6000 :  7.378999999999959
7000 :  7.511999999999966
8000 :  7.388999999999959
9000 :  7.471999999999961
10000 :  7.375999999999965


In [23]:
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[ 0.          0.          0.          0.          0.          0.        ]
 [ 2.46810726  1.02853031  1.69518577 -2.85338392  9.6220697  -4.18283343]
 [ 9.01010468  4.15363232  6.82146892  9.55553998 14.11880599  1.03650442]
 ...
 [-0.78295732  0.84603199 -0.83151088  5.55285716 -4.51046037 -4.1971362 ]
 [-2.03272021 -0.33638664 -2.49883597  2.83428751 -4.30641753 -7.52622502]
 [ 1.28638332  1.31172739 -0.1        12.27888007 -1.86117362 -1.15707714]]


## Test the agent

In [24]:
for episode in range(3):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 20:
                print("****Successful dropoff!****")
                time.sleep(3)
            else:
                print("****Fail!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        
        state = new_state

env.close()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
****Successful dropoff!****
